# Fetch Data

In [1]:
import pandas as pd
from warnings import filterwarnings
filterwarnings("ignore")

A = open("C:/Users/Anuj.kumar/Desktop/sherlock.txt",encoding="utf")

In [2]:
W=A.read()

In [3]:
W = W[0:20000]

# Preprocessing

In [4]:
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

ps = PorterStemmer()
lm = WordNetLemmatizer()

words = []
for i in W.split(" "):
    upr = i.upper()
    splch = re.sub("[^A-Z0-9]","",upr)
    stemmed = ps.stem(splch)
    lem = lm.lemmatize(stemmed)
    words.append(lem.upper())

# Convert Data in X|Y format

In [5]:
str_X = []
str_Y = []

In [6]:
for i in range(3,len(words),1):
    fw = words[i-3]
    sw = words[i-2]
    tw = words[i-1]
    fnw = words[i]
    str_X.append(str(fw) + " " + str(sw) + " " + str(tw))
    str_Y.append(str(fnw))

In [7]:
import pandas as pd
Q = pd.DataFrame([str_X,str_Y]).T
Q.columns = ["X","Y"]

In [8]:
Q.head()

,X,Y
0,PROJECT GUTENBERG THE,ADVENTUR
1,GUTENBERG THE ADVENTUR,OF
2,THE ADVENTUR OF,SHERLOCK
3,ADVENTUR OF SHERLOCK,HOLM
4,OF SHERLOCK HOLM,BY


# Converting Data to ARRAY | TENSOR

Find out unique words

In [9]:
import numpy as np
uwords =np.unique(words)

In [10]:
len(uwords)

1240

In [11]:
len(str_X)

3268

Creation of blank arrays as per X and Y sizes

In [12]:
import numpy as np
X_arr = np.zeros((len(str_X),len(uwords),3),dtype=bool)
Y_arr = np.zeros((len(str_X),len(uwords)),dtype=bool)

Create a dictionary containing words and its position

In [13]:
word_position_finder = {}
for i in range(0,len(uwords),1):
    word_position_finder[uwords[i]] = i

In [14]:
for i,j in enumerate(str_X):
    for j,k in enumerate(j.split(" ")):
        pos = word_position_finder[k]
        X_arr[i,pos,j]=1

In [15]:
for i,j in enumerate(str_Y):
    pos = word_position_finder[j]
    Y_arr[i,pos]=1

In [16]:
Y_arr

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

# Create a NN

In [17]:
from keras.layers import Dense,RNN,LSTM
from keras.models import Sequential

In [18]:
model = Sequential()
model.add(LSTM(200,input_shape=(len(uwords),3)))
model.add(Dense(len(uwords),activation="softmax"))

In [19]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(X_arr,Y_arr,batch_size=1000,epochs=3)

Epoch 1/3
4/4 [==============================] - 131s 29s/step - loss: 7.1217 - accuracy: 0.0352
Epoch 2/3
4/4 [==============================] - 98s 22s/step - loss: 7.1045 - accuracy: 0.0453
Epoch 3/3
4/4 [==============================] - 101s 24s/step - loss: 6.7673 - accuracy: 0.0453


,X,Y
0,PROJECT GUTENBERG THE,ADVENTUR
1,GUTENBERG THE ADVENTUR,OF
2,THE ADVENTUR OF,SHERLOCK
3,ADVENTUR OF SHERLOCK,HOLM
4,OF SHERLOCK HOLM,BY
...,...,...
3263,TOOK A NOTE,OF
3264,A NOTE OF,IT
3265,NOTE OF IT,ONE
3266,OF IT ONE,OTHER


# Prepare input data for prediction

In [20]:
input_X_arr = np.zeros((1,len(uwords),3),dtype=bool)

In [21]:
sent = input("Enter a sentence [3 words only]: ")

Enter a sentence [3 words only]: GUTENBERG THE ADVENTUR


In [23]:
for j,k in enumerate(sent.split(" ")):
    pos = word_position_finder[k]
    input_X_arr[0,pos,j]=1

In [24]:
R = pd.DataFrame([uwords,model.predict(input_X_arr)[0]]).T
R.columns= ["word","prob"]

In [25]:
pred = R.sort_values(by="prob",ascending=False).head(1).word.values[0]

In [26]:
print("Predicted value is --> ",pred)

Predicted value is -->  THE
